In [114]:
import pandas as pd
import os
import sys
import json
import re

In [115]:
# Add repo
git_dir = os.path.abspath('../')
sys.path.append(os.path.join(git_dir, 'lib', 'GoEmotions-pytorch') )
sys.path.append(os.path.join(git_dir, 'lib', 'utils') )


In [3]:
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline
import re
from tqdm import tqdm
import numpy as np 
import pandas as pd
import pickle

In [35]:
# Add repo
git_dir = os.path.abspath('../')
sys.path.append(os.path.join(git_dir, 'lib', 'utils') )

# Define data output path
transcript_path = os.path.join(git_dir, 'data', 'pm-transcripts-processed/aggregated-spellfixed-typed.csv')
df = pd.read_csv(transcript_path)

In [36]:
df.head()

,Unnamed: 0,pm,date,type,content,sentences
0,0,"Abbott, Tony",24/08/2014,Media Release,Every day the Government is working to build a...,['every day the government is working to build...
1,1,"Gorton, John",14/03/1968,Statement in Parliament,"STATEMENT BY THE PRIME MINISTER, THE RT. HON....",['the house of representatives on a of march o...
2,2,"Turnbull, Malcolm",17/11/2016,Transcript,PRIME MINISTER: This is a great initiative. Wh...,['what you re seeing here is real enthusiasm f...
3,3,"Fraser, Malcolm",27/09/1978,Media Release,"FOR PRESS 27 SEPTEMBER, 1978 GRANT TO WORLD W...",['for press of september of of grant to world ...
4,4,"Howard, John",24/03/2004,Interview,"JOURNALIST: Prime Minister, is Mark Latham pla...",['journalist prime minister is mark latham pla...


In [59]:
df_speeches = df[((df['type']=='Speech') | (df['type']=='Media Release'))]

In [63]:
def batch_gen(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [77]:
tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")
model = model.to('cuda:0')

goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3,
    device=0
)


processed_sentence_lists = []
for ind, sentence_list_repr in enumerate(tqdm(df_speeches['sentences'])):
    processed_sentences = []
    
    sentence_list_raw = eval(sentence_list_repr)
    sentence_list = []
    
    for sentence_ind, sentence in enumerate(sentence_list_raw):
        tokens = tokenizer(sentence)
        if len(tokens['input_ids'])>500:
            print('Sentence %d in document %d is too long'%(sentence_ind, ind))
        else:
            sentence_list.append(sentence)
    
    for batch in batch_gen(sentence_list, n=32):
        processed_sentences.extend(goemotions(batch))
        #processed_sentences.append(goemotions(sentence)[0])
            #ts['emotions'].append(goemotions(sentence)[0])
    processed_sentence_lists.append(processed_sentences)
        

  7%|▋         | 910/13674 [01:05<17:24, 12.22it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (636 > 512). Running this sequence through the model will result in indexing errors


Sentence 911 in document 23 is too long


 11%|█         | 1513/13674 [01:49<17:46, 11.40it/s]

Sentence 1512 in document 20 is too long


 11%|█         | 1528/13674 [01:51<21:13,  9.54it/s]

...

 98%|█████████▊| 13454/13674 [16:56<00:10, 20.86it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (672 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (832 > 512). Running this sequence through the model will result in indexing errors


Sentence 13454 in document 28 is too long
Sentence 13454 in document 30 is too long


100%|█████████▉| 13666/13674 [17:12<00:00, 20.04it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (604 > 512). Running this sequence through the model will result in indexing errors


Sentence 13667 in document 16 is too long


100%|██████████| 13674/13674 [17:12<00:00, 13.24it/s]


In [89]:
df_speeches_processed = df_speeches.copy()

In [90]:
df_speeches_processed['sentences_emotions'] = processed_sentence_lists
#processed_sentence_lists[0]

In [91]:
df_speeches_processed.head()

,Unnamed: 0,pm,date,type,content,sentences,sentences_emotions
0,0,"Abbott, Tony",24/08/2014,Media Release,Every day the Government is working to build a...,['every day the government is working to build...,"[{'labels': ['approval', 'optimism', 'realizat..."
3,3,"Fraser, Malcolm",27/09/1978,Media Release,"FOR PRESS 27 SEPTEMBER, 1978 GRANT TO WORLD W...",['for press of september of of grant to world ...,"[{'labels': ['neutral'], 'scores': [0.9980432]..."
8,8,"Fraser, Malcolm",21/07/1976,Media Release,FOR PRESS 21 July 1976 SENATOR COTTON TO LEAD...,['for press of july of of senator cotton to le...,"[{'labels': ['neutral'], 'scores': [0.9990237]..."
9,9,"Gillard, Julia",25/08/2011,Media Release,Patients and families in regional communities...,['patients and families in regional communitie...,"[{'labels': ['approval', 'neutral'], 'scores':..."
12,12,"Hawke, Robert",06/06/1986,Media Release,PRIMEST AINIE FOR MEDIA 6 JUNE 1986 I am plea...,['priest annie for media a june of of i am ple...,"[{'labels': ['neutral'], 'scores': [0.9751662]..."


In [97]:
# Convert string repr of sentences dicts back into dicts
df_speeches_processed['sentences'] = df_speeches_processed['sentences'].apply(lambda x: eval(x))

In [100]:
df_speeches_processed.to_pickle( os.path.join(git_dir, 'data', 'pm-transcripts-processed', 'aggregated-spellfixed-typed-emotions.pkl') )

In [101]:
text = "thank you or bartholomew reverend summer my parliamentary colleagues irish draper chris pine and joe scale or mayor ladies and gentlemen. its a beautiful adelaide morning and its a great pleasure to be with you and to share a few moments in opening these new facilities which are the product of wonderful work of lutheran aged care here in south australia. let me say how much i admire and the government admires the contribution that over the years church based aged care facilities have made to the caring of the aged in australia. they have in so many ways been the example that has been followed by others and i hope for years into the future whenever we think in australia of aged care facilities we think automatically of the contribution that has been made by so many of the churches and other religiously connected organisations to the care of the aged in australia. in common with all other societies that are like australia we are an ageing country. the number of people over the age of of will grow as the years go by and the proportion of people of working age will diminish. this raises a number of challenges for governments well into the future. one part of that challenge will of course be in the area of aged care facilities. it is the philosophy of our government that we should assist people to remain in their own homes as long as is possible and indeed the great bulk of people are able to realise that goal. we provide through the community care packages increased facilities to assist people to go on living for longer periods of time in their own homes. but we also recognise that there will come a point for many people where that is no longer either possible or as a matter of choice desirable. and therefore the other element is to provide the appropriate levels of support and the appropriate incentives for the maintenance of a network of aged care facilities of retirement villages of hostels and of nursing homes. and there is no better environment in which to provide those facilities than an environment which is underpinned by the care and the compassion and the commitment through their common faith of religious based organisations. and lutheran homes are a wonderful example of that and i want to thank you again for the contribution that you have made over the years. quite rightly your chairman pointed to the governments examination of the hogan review into the pricing and other aspects of aged care homes. it is an issue which is in front of the government literally at the moment and i can assure you that the recommendations of that committee and the importance that is placed by that committee on the maintenance of proper and reasonable levels of funding from the commonwealth government will be examined very carefully. we regard the care of the aged the proper looking after of people who have given so much to this country as i look out on this audience i think of a truly heroic generation many of whom went through a depression and a world war world war ii were part of the great post world war rebuilding of this country when we welcomed millions of people from other parts of the world and together all of us created the stable harmonious modern australia we owe your generation a great debt and we will not walk away from that responsibility. we have of course as a government responsibilities to many sectors of society. i did an interview this morning and in the space of about of minutes on radio i was asked when are you going to cut tax on this that and the other and i was of course asked when are you going to spend some more money and of course you all keep the economy ticking over well and the budget in balance. you just have to juggle a few balls in the air and its extremely easy. we can never satisfy everybody completely all of the time. we have to make judgements we have to make choices but i can assure you that we will conscientiously address the needs of the aged care sector. can i just finally say if i may be permitted to do so at such a gathering as this to speak of the personal qualities of your two federal representatives who are here today. in fact you have had some very famous people who have worked here over the years. and she brings to her parliamentary duties because of her background in caring for people a great understanding and also her commitment to our services. and chris pine in whose electorate this home is located i also commend to you to all of you his great energy and his great commitment and his concern to look after the interests of everybody in the community. i'm not leaving you out joe but you know they re just my federal mates and i wanted to say something about them. he a a hard working bloke too and i wouldn't want anybody to forget that. but ladies and gentlemen its great to share such a lovely morning with you. i started off this morning going for a walk along the torrens river and as you know i do a lot of walking everywhere in the world and there in to a nicer part of australia in which to go for a walk than that beautiful torrens river. i can see the adelaide oval out of the corner of my eye and that sort of keeps me very very happy. and i have great pleasure in declaring these new facilities well and truly open and long may you all live peacefully and happily in a supportive christian environment in these wonderful facilities"

## Repeat using Ekman Taxonomy

In [119]:
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline
from pprint import pprint

tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-ekman")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-ekman")
model = model.to('cuda:0')

goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3,
    device=0
)

In [120]:
df_speeches = df[((df['type']=='Speech') | (df['type']=='Media Release'))]

In [121]:
processed_sentence_lists = []
for ind, sentence_list_repr in enumerate(tqdm(df_speeches['sentences'])):
    processed_sentences = []
    
    sentence_list_raw = eval(sentence_list_repr)
    sentence_list = []
    
    for sentence_ind, sentence in enumerate(sentence_list_raw):
        tokens = tokenizer(sentence)
        if len(tokens['input_ids'])>500:
            print('Sentence %d in document %d is too long'%(sentence_ind, ind ))
        else:
            sentence_list.append(sentence)
    
    for batch in batch_gen(sentence_list, n=32):
        processed_sentences.extend(goemotions(batch))
        #processed_sentences.append(goemotions(sentence)[0])
            #ts['emotions'].append(goemotions(sentence)[0])
    processed_sentence_lists.append(processed_sentences)
        

  7%|▋         | 910/13674 [01:03<17:07, 12.42it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (636 > 512). Running this sequence through the model will result in indexing errors


Sentence 911 in document 23 is too long


 11%|█         | 1513/13674 [01:47<17:24, 11.65it/s]

Sentence 1512 in document 20 is too long


 11%|█         | 1528/13674 [01:48<20:54,  9.68it/s]

...

 98%|█████████▊| 13454/13674 [16:45<00:10, 20.95it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (672 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (832 > 512). Running this sequence through the model will result in indexing errors


Sentence 13454 in document 28 is too long
Sentence 13454 in document 30 is too long


100%|█████████▉| 13666/13674 [17:00<00:00, 20.14it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (604 > 512). Running this sequence through the model will result in indexing errors


Sentence 13667 in document 16 is too long


100%|██████████| 13674/13674 [17:01<00:00, 13.39it/s]


In [123]:
df_speeches_processed = df_speeches.copy()
df_speeches_processed['sentences_emotions'] = processed_sentence_lists

# Convert string repr of sentences dicts back into dicts
df_speeches_processed['sentences'] = df_speeches_processed['sentences'].apply(lambda x: eval(x))

# Save
df_speeches_processed.to_pickle( os.path.join(git_dir, 'data', 'pm-transcripts-processed', 'aggregated-spellfixed-typed-emotions-ekman.pkl') )